# Gaussian processes – spatial regression (Python version)
GMM, INSA Toulouse, France <br />
Andrés F. López-Lopera, ONERA-DTIS <br />
May 2021
<br />
___

For this lab session, you are free to use the language of your choice (e.g. R or Python). In this notebook we propose Python implementations.

In [ ]:
# loading useful toolboxes
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

plt.rc('text', usetex=True)
%matplotlib inline

## 2D spatial GP regression

In this practical session, we consider spatial data. We generate first some 2D synthetic data from a centred spatial GP with covariance function given by:

$$k(\textbf{x},\textbf{y}) = \sigma^2 \exp\left( - \frac{(x_1-y_1)^2}{2 \theta_1^2} - \frac{(x_2-y_2)^2}{2 \theta_2^2}\right),$$

with $\textbf{x} = (x_1, x_2) \in \mathbb{R}^2$ and $\textbf{y} = (y_1, y_2) \in \mathbb{R}^2$.

**Question 1.** For a grid of 30x30 points on $[0, 1]^2$, simulate a GP realisation using the function ``np.random.multivariate_normal`` and considering the variance parameter $\sigma^2 = 1$ and the length-scale parameters $\theta_1 = \theta_2 = 0.2$. Display the resulting 2D SE kernel and the generated GP sample (2D and 3D visualisation).

In [ ]:
# generating synthetic data from a GP
def SEkernel2D(x, y, param): # 2D squared exponential kernel
    """ 2D Squared exponential kernel
    input:
      x,y: input vectors
      param: parameters (sigma,theta1,theta2)
    output:
      covariance matrix cov(x,y)
    """
    sigma2, theta1, theta2 = param[0], param[1], param[2]
    dist1 = # to complete
    dist2 = # to complete
    return sigma2*np.exp(-0.5*(dist1**2 + dist2**2))

n = 30 # number of points per input dimension
n_total = n**2 # total number of points
x = np.linspace(0, 1, n) 
x2, x1 = np.meshgrid(x,x) # generating an equispaced 2D grid
X = np.concatenate((x1.reshape(-1,1), x2.reshape(-1,1)), axis=1) # 2D grid
param = [1, 0.2, 0.2] # covariance parameters
kern = # to complete

# sampling from the resulting GP
np.random.seed(0) # a seed for reproducibility
y = # to complete

# plotting the results
fig = plt.figure(figsize=(18,5))
ax = fig.add_subplot(131)
plt.contourf(kern)
plt.xlabel("$x$"); plt.ylabel("$x'$"); plt.title("2D SE kernel")

ax = fig.add_subplot(132, projection='3d')
ax.plot_surface(x, x, y.reshape(n, n), rstride=1, cstride=1,
                edgecolor='none', alpha=0.3)
plt.xlim(-0.1, 1.1), plt.ylim(-0.1, 1.1)
ax.set_xlabel(r'$x_1$'); ax.set_ylabel(r'$x_2$'); ax.set_zlabel(r'$y$');
ax.set_title("GP sample - 3D visualisation")

ax = fig.add_subplot(133)
cf = ax.pcolormesh(x, x, y.reshape(n, n), shading='auto')
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_title(r'$y$ (2D visualisation)')
fig.colorbar(cf, ax=ax);

**Question 2.** Repeat the procedure in **Question 1** but changing the values of $\sigma^2, \theta_1$ and $\theta_2$. What can you observe?

**Question 3.** For instance, consider $\sigma^2 = 1$ and $\theta_1 = \theta_2 = 0.2$. In real-world applications, we commonly have acces to a few amount of training data and we aim at predicting values at new points. Split the data in two subsets: one containing the training data and the other one containing the test data. Use the command ``np.random.permutation`` in order to generate a random permutation with ``n_total`` points. Then, take the first 1% of indexes of the permutation for constructing the training set. The remaining indexes are used for building up the test data. 

In [ ]:
## splitting data into a training set and a test set
# generating a random permutation with n_total points
np.random.seed(2)
idx = # to complete

# taking a proportion of the permuted indexes for defining the training dataset
prop_train = 0.01
n_train = int(np.ceil(prop_train * n_total))
X_train = X[idx[:n_train], :]
y_train = y[idx[:n_train]]

# using the remaining indexes for constructing the test dataset
X_test = X[idx[n_train:], :]
y_test = y[idx[n_train:]]

**Question 4.** With the help of the ``KRG`` function from the ``smt`` toolbox, construct a GP regression model (also known as a Kriging model in geostatistics) using a squared exponential kernel function. You can see the documentation in https://smt.readthedocs.io/en/latest/_src_docs/surrogate_models/krg.html for further details.

In [ ]:
from smt.surrogate_models import KRG

# defining the kriging model
sm = # to complete
# passing the training data to the KRG model
sm.set_training_values(X_train, y_train)
# training the model
sm.train()

**Question 5.** Use the resulting GP model to predict values at some points: either at the training points or at the test points. Compute an error criterion based on the MSE for both prediction results. What can you conclude?

In [ ]:
# predictions at new points
pred_train = # to complete
pred_test = # to complete

In [ ]:
rmse_train = # to complete
rmse_test = # to complete
print("RMSE - training data:", rmse_train)
print("RMSE - test data:", rmse_test)

fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(121)
ax.plot(y_train, pred_train, ".")
ax.plot([np.min(y_train), np.max(y_train)],
        [np.min(pred_train),np.max(pred_train)], "--")
ax.set_xlabel("true values"); ax.set_ylabel("prediction")
ax.set_title("training data - RMSE: %.3f" % rmse_train)

ax = fig.add_subplot(122)
ax.plot(y_test, pred_test, ".")
ax.plot([np.min(y_test), np.max(y_test)],
        [np.min(pred_test),np.max(pred_test)], "--")
ax.set_xlabel("true values")
ax.set_ylabel("prediction")
ax.set_title("test data - RMSE: %.3f" % rmse_test);

**Question 6.** Use the resulting GP model to predict values at the entire domain $[0,1]^2$. Display 2D and 3D visualisations of the target profile, the predicted GP mean and the predictive GP variance.

In [ ]:
pred_all = # to complete
var_all = # to complete

fig = plt.figure(figsize=(18,10))
ax = fig.add_subplot(131, projection='3d')
ax.plot_surface(x, x, y.reshape(n,n), rstride=1, cstride=1,
                edgecolor='none', alpha=0.3)
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_zlabel(r'$y$');

ax = fig.add_subplot(132, projection='3d')
ax.plot_surface(x, x, pred_all.reshape(n,n), rstride=1, cstride=1,
                edgecolor='none', alpha=0.3, color="C1")
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_zlabel("predictive mean")


fig = plt.figure(figsize=(18,5))
ax = fig.add_subplot(131)
cf = ax.pcolormesh(x, x, y.reshape(n,n), shading='auto')
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_title(r'$y$')
fig.colorbar(cf, ax=ax);

ax = fig.add_subplot(132)
cf = ax.pcolormesh(x, x, pred_all.reshape(n,n), shading='auto')
ax.plot(X_train[:,0], X_train[:,1], "k.")
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_title("predictive mean")
fig.colorbar(cf, ax=ax);

ax = fig.add_subplot(133)
cf = ax.pcolormesh(x, x, var_all.reshape(n,n), shading='auto')
ax.plot(X_train[:,0], X_train[:,1], "k.")
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_title("predictive variance")
fig.colorbar(cf, ax=ax);

**Question 7.** Repeat the procedure from **Questions 1-6** but changing the covariance function, the covariance parameters $\sigma^2, \theta_1, \theta_2$, the seed of the the randomState, the percentage of training data, etc. What can you conclude?

**Bonus.** Using a 2D dataset of your choice (see https://archive.ics.uci.edu/ml/datasets.php for some examples), repeat the procedure from **Questions 3-6**.